In [1]:
import numpy as np
from random import random

### Class MTLine
An object of this class represents one location on the axon cross section containing multiple microtubules along the full length of the axon model. <br>

<b> Fields: </b> <br>
mts: list of Microtubule objects, all microtubules with the same x,y coordinates <br>
x: float, x coordinate of the microtubule line <br>
y: float, y coordinate of the microtubule line <br>

In [2]:
class MTLine:
    def __init__(self):
        self.mts = []
        self.x = None
        self.y = None

### Class Microtubule
An object of this class represents one microtubule in the image stack. <br>

<b> Fields: </b> <br>

nodes: list of Node objects, holds all the nodes correponding to this microtubule <br>
zstart: float, z coordinate of the first node in the microtubule <br>
zend: float, z coordinate of the last node in the microtubule <br>
numCL: int, number of crosslinks attached to this microtubule

In [3]:
class Microtubule:
    def __init__(self):
        self.nodes = []
        self.zstart = None
        self.zend = None
        self.numCL = 0

    def add_node(self, node):
        self.nodes.append(node)
        
    def add_cl(self):
        self.numCL = self.numCL + 1 

### Class Node
An object of this class represents one node in the finite element model. <br>

<b> Fields: </b> <br>
coords: list of floats, xyz coordinates of the node <br>
previous: Node object, corresponds to the previous node belonging to the same microtubule <br>
next: Node object, corresponds to the next node belonging to the same microtubule <br>
id: int, unique identifying number to reference this Node object <br>
connected: boolean, True if this node is connected to any other node <br>
mt: Microtubule object, refers to the microtubule to which this node belongs <br>

In [4]:
class Node:
    def __init__(self, coords):
        self.coords = coords
        self.previous = None
        self.next = None
        self.id = None
        self.connected = False
        self.mt = None
    
    def set_id(self, index):
        self.id = index
        
    def set_mt(self, mt):
        self.mt = mt

## Main Code
The following code generates an idealized geometry of evenly spaced microtubules and randomly generated crosslinks. The resulting finite element mesh is output in an ABAQUS input file.

In [5]:
#model parameters
n = 5 #number of rings
r = 45 #nm - microtubule spacing
rConn = 50 #nm - maximum crosslink length
lMTMax = 8000 #nm - microtubule length
lGap = 50 #nm - gap length between adjacent microtubules in the same microtubule line
lAxon = 15000 #nm - total axon length
thLink = 0 #deg - crosslink angle
dlMT = 50 #nm - microtubule element size

In [ ]:
#determining the x,y coordinates for all microtubule lines
xMT = [0.]
yMT = [0.]

for i in range(n+1):
    
    if i%2 == 0 and i!=0:
        xMT.extend([0., 0.])
        yMT.extend([i*np.sqrt(3)*r/2, -i*np.sqrt(3)*r/2])
        
    for j in range((2*n-(i-1))//2):
        if i==0:
            xMT.extend([-(j+1)*r,(j+1)*r])
            yMT.extend([0., 0.])
        elif i%2==1:
            xMT.extend([-(j+0.5)*r, -(j+0.5)*r, (j+0.5)*r, (j+0.5)*r])
            yMT.extend([i*np.sqrt(3)*r/2, -i*np.sqrt(3)*r/2, i*np.sqrt(3)*r/2, -i*np.sqrt(3)*r/2])
        else:
            xMT.extend([-(j+1)*r, -(j+1)*r, (j+1)*r, (j+1)*r])
            yMT.extend([i*np.sqrt(3)*r/2, -i*np.sqrt(3)*r/2, i*np.sqrt(3)*r/2, -i*np.sqrt(3)*r/2])
            
mtlines = []
for i in range(len(xMT)):
    mtlines.append(MTLine())
    mtlines[-1].x = xMT[i]
    mtlines[-1].y = yMT[i]

#determining possible crosslink locations
connCS = []
for i in range(len(yMT)):
    for j in range(i+1, len(yMT)):
        dist = np.sqrt((xMT[j]-xMT[i])**2+(yMT[j]-yMT[i])**2)
        if(dist<rConn):
            connCS.append([i,j,dist])
            connCS.append([j,i,dist])
nCS = len(connCS)

#determining initial microtubule lengths for the most proximal microtubules
lMT0 = np.linspace(0, lMTMax+lGap, len(yMT)+1).astype(int)
lMT0 = np.delete(lMT0, 0)
np.random.shuffle(lMT0)

#determining z coordinates for the remaining microtubules
zMT = []
for i in range(len(yMT)):
    z0 = 0.
    z1 = lMT0[i]
    zMT.append([[z0,z1,0]])
    
    doContinue = True
    while(doContinue):
        z0 = z1 + lGap
        z1 = z0+lMTMax
        
        if(z0>lAxon):
            doContinue = False
        else:
            if(z1>lAxon):
                doContinue = False
                z1 = lAxon
            zMT[i].append([z0,z1,0])
            
#calculating total microtubule length for crosslink density calculation
totalLen = 0
for i in range(len(zMT)):
    for j in range(len(zMT[i])):
        length = zMT[i][j][1]-zMT[i][j][0]
        totalLen = totalLen + length

#store microtubule locations
for index, mtl in enumerate(zMT):
    for mt in mtl:
        mtlines[index].mts.append(Microtubule())
        mtlines[index].mts[-1].zstart = mt[0]
        mtlines[index].mts[-1].zend = mt[1]

#generate crosslinks
dlLink = 15000./((totalLen/1000.)*30.)
crosslinks = []
for i in range(1, int(lAxon/dlLink)):
    z0 = i*dlLink
    csID = np.random.randint(0,nCS)
    z1 = z0+connCS[csID][2]*np.tan(thLink)
    
    zMT0 = np.asarray(zMT[connCS[csID][0]])
    zMT1 = np.asarray(zMT[connCS[csID][1]])
    
    try:
        id01 = [n for n,i in enumerate(zMT0[:,0]) if i<=z0][-1] 
        id02 = [n for n,i in enumerate(zMT0[:,1]) if i>=z0][0]
        
        id11 = [n for n,i in enumerate(zMT1[:,0]) if i<=z1][-1]
        id12 = [n for n,i in enumerate(zMT1[:,1]) if i>=z1][0]
        
        if (id01==id02 and id11==id12):
            crosslinks.append([connCS[csID][0], id01, connCS[csID][1], id11, z0, z1])
            
            zMT[connCS[csID][0]][id01][2] += 1
            zMT[connCS[csID][1]][id11][2] += 1
    except:
        pass

#create finite element mesh
nodesAll = []
elementsMT = []
elementsCL = []

nodeID = 1
elementID = 1

#microtubule elements
for mtlID in range(len(mtlines)):
    for mtID in range(len(mtlines[mtlID].mts)):
        z0 = mtlines[mtlID].mts[mtID].zstart
        z1 = mtlines[mtlID].mts[mtID].zend
        
        nelMT = np.ceil((z1-z0)/dlMT).astype(int)
        nnodMT = nelMT + 1
        
        zn = np.concatenate([np.linspace(z0,z0+(nelMT-1)*dlMT,nelMT), z1*np.ones(1)])
        
        if (len(zn)<2 or np.abs(zn[-1]-zn[-2])<1e-6):
            zn = np.delete(zn,[len(zn)-2])
            nelMT -= 1
            nnodMT -= 1
            
        for i in range(nnodMT):
            mtlines[mtlID].mts[mtID].nodes.append(Node((mtlines[mtlID].x, mtlines[mtlID].y, zn[i])))
            mtlines[mtlID].mts[mtID].nodes[-1].set_id(nodeID)
            nodeID += 1
            
            if i>0:
                elementsMT.append((elementID, mtlines[mtlID].mts[mtID].nodes[-2].id, 
                                   mtlines[mtlID].mts[mtID].nodes[-1].id))
                elementID += 1

#crosslink elements
for csid in range(len(crosslinks)):
    mtl0 = crosslinks[csid][0]
    mt0 = crosslinks[csid][1]
    mtl1 = crosslinks[csid][2]
    mt1 = crosslinks[csid][3]
    z0 = crosslinks[csid][4]
    z1 = crosslinks[csid][5]
    
    zmt0 = zMT[mtl0][mt0][0]
    zmt1 = zMT[mtl1][mt1][0]
    
    nid0 = mtlines[mtl0].mts[mt0].nodes[0].id + np.round((z0-zmt0)/dlMT).astype(int)
    nid1 = mtlines[mtl1].mts[mt1].nodes[0].id + np.round((z1-zmt1)/dlMT).astype(int)
    
    elementsCL.append((elementID, nid0, nid1))
    elementID += 1

In [ ]:
#write ABAQUS input file
# Abaqus units: length - nm, force - nN, time - s, stress - GPa

f = open("idealizedModel.inp", "w+")
f.write("*HEADING \n")
f.write("model with idealized geometry of 5 rings, 91 microtubules per cross section, 8um MT length, 30 XL density \n")
f.write("SI Units \n")
f.write("**\n** Model definition \n** \n")

f.write("*NODE, NSET=NSET_ALL\n") #node definitions

for node in nodesAll:
    f.write(str(node.id) + ", " + str(node.coords)[1:-1] + '\n')

RPid = len(nodesAll)+1
f.write(str(RPid) + ", 0.0, 0.0, 15100.0 \n") # add a node for the reference point

f.write("*ELEMENT, TYPE=B33, ELSET=ELSET_MT \n") #microtubule elements

for el in elementsMT:
    f.write(str(el)[1:-1] + '\n')
    
f.write("*ELEMENT, TYPE=T3D2, ELSET=ELSET_CL \n") #crosslink elements

for el in elementsCL:
    f.write(str(el)[1:-1] + '\n')

#create node sets for boundary conditions and output requests    
f.write("*NSET, NSET=NSET_REFPOINT\n")
f.write(str(RPid)+ '\n')
        
fixedEndNodes = []

for node in nodesAll:
    if node.coords[2]==0.0:
        fixedEndNodes.append(node.id)

#figure out how many lines we need in the input file (max 16 nodes per line)
numlines = len(fixedEndNodes)//16 
if len(fixedEndNodes) % 16 != 0:
    numlines = numlines + 1  
    
f.write("*NSET, NSET=NSET_FIXEDEND\n")

for i in range(numlines):
    for index, node in enumerate(fixedEndNodes[16*i:16*(i+1)]):
        if index != len(fixedEndNodes[16*i:16*(i+1)]) - 1:
            f.write(str(node) + ', ')
        else:
            f.write(str(node) + '\n')
            
loadNodes = []
allOtherNodes = []
for node in nodesAll:
    if node.coords[2]==lAxon:
        loadNodes.append(node.id)
    else:
        allOtherNodes.append(node.id)
        
numlines = len(loadNodes)//16
if len(loadNodes) % 16 != 0:
    numlines = numlines + 1 

f.write("*NSET, NSET=NSET_LOADEND\n")

for i in range(numlines):
    for index, node in enumerate(loadNodes[16*i:16*(i+1)]):
        if index != len(loadNodes[16*i:16*(i+1)]) - 1:
            f.write(str(node) + ', ')
        else:
            f.write(str(node) + '\n')
            
numlines = len(allOtherNodes)//16
if len(allOtherNodes) % 16 != 0:
    numlines = numlines + 1
    
f.write("*NSET, NSET=NSET_ALLBUTLOAD\n")

for i in range(numlines):
    for index, node in enumerate(allOtherNodes[16*i:16*(i+1)]):
        if index != len(allOtherNodes[16*i:16*(i+1)]) - 1:
            f.write(str(node) + ', ')
        else:
            f.write(str(node) + '\n')
            
f.write("*SURFACE, TYPE=NODE, NAME=COUPLING_NODES\n")    
f.write("NSET_LOADEND\n")

f.write("*BEAM SECTION, ELSET=ELSET_MT, SECTION=CIRC, MATERIAL=MT \n")
f.write("11.28\n")
f.write("-1.0, 0.0, 0.0\n")

f.write("*SOLID SECTION, ELSET=ELSET_CL, MATERIAL=CL \n")
f.write("1.0\n")

#Material definitions
f.write("*MATERIAL, NAME=MT\n")
f.write("*ELASTIC \n")
f.write("1.2, 0.3\n")

f.write("*MATERIAL, NAME=CL\n")
f.write("*ELASTIC \n")
f.write("0.01, 0.3\n")
        
#coupling to reference point
f.write("*COUPLING, CONSTRAINT NAME=load_constraint, REF NODE=NSET_REFPOINT, SURFACE=COUPLING_NODES\n")
f.write("*KINEMATIC\n")
f.write("1, 3\n")

#fix nodes at one end
f.write("*BOUNDARY\n")
f.write("NSET_FIXEDEND, 3\n")

#fix rotations
f.write("NSET_ALL, 4, 6\n")

#fix reference point lateral translation
f.write("NSET_REFPOINT, 1, 2\n")

#add load step
f.write("*STEP, NAME=LOADSTEP, NLGEOM=YES\n")
f.write("*STATIC\n")
f.write("0.01, 1, 10E-5\n")

f.write("*BOUNDARY, TYPE=DISPLACEMENT\n")
f.write("NSET_REFPOINT, 3, 3, 250.\n")

#output
f.write("*OUTPUT, FIELD\n")
f.write("*NODE OUTPUT\n")
f.write("RF, U\n")
f.write("*ELEMENT OUTPUT, DIRECTIONS=YES\n")
f.write("EE, ELEDEN, NE, S\n")
f.write("*OUTPUT, HISTORY\n")
f.write("*NODE OUTPUT, NSET=NSET_REFPOINT\n")
f.write("RF3\n")

f.write("*END STEP")

f.close()